In [ ]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [ ]:
#!git clone https://github.com/jthickstun/watermark
!pip install -q transformers PyExecJs

  Preparing metadata (setup.py) ... done


In [ ]:
cd '/drive/My Drive/Colab Notebooks/ISO/'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO


In [ ]:
cd kuditipudi-watermark

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/kuditipudi-watermark


In [ ]:
!pip install -q -r new_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 54.4 MB/s eta 0:00:00


In [ ]:
cd demo

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/kuditipudi-watermark/demo


In [ ]:
import os, argparse

import torch
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import execjs

In [ ]:
with open('detect.js', 'r') as f_1:
    with open('mersenne.js', 'r') as f_2:
        files_code = "\n".join([line for line in f_2.readlines() if not line.startswith("export")])
        files_code += "\n\n"
        files_code += "\n".join([line for line in f_1.readlines() if not line.startswith("import") and not line.startswith("window")])

In [ ]:
checkpoint = 'facebook/opt-1.3b'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [ ]:
cd '../../instances'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/instances


In [ ]:
def obtain_p_value(tokens, key=42, n=256, vocab_size=50265):
    js_code = files_code + "\n\n"
    # Change the line   const vocab_size = 32000 to const vocab_size = {vocab_size}
    js_code = js_code.replace(" const vocab_size = 32000", f" const vocab_size = {vocab_size}")
    js_code += f"var result = permutation_test({tokens}, {key}, {n}, {len(tokens)});"
    # Use execjs to execute the JavaScript code
    ctx = execjs.compile(js_code)
    # Get the result of the JavaScript code
    return ctx.eval('result')


def calculate_scores(filename, keyword, custom_filename=None, threshold=0.01):
    detection_filename = (filename[:-5] + '_detection.json') if custom_filename == None else custom_filename
    df = pd.read_json(filename, lines=True)
    detected = 0
    not_detected = 0
    with open(detection_filename, "r+" if os.path.exists(detection_filename) else "w+") as f:
        with torch.no_grad():
            for i in tqdm(range(len(f.readlines()), len(df["input"]))):
                line = (df["input"][i] if keyword == "continuation" else "") + df[keyword][i]

                gen_tokens = tokenizer.encode(line, return_tensors='pt', truncation=True, max_length=256).numpy()[0]
                p_value = obtain_p_value(gen_tokens.tolist(), key=42, n=256, vocab_size=len(tokenizer))
                wm_pred = 1 if p_value <= threshold else 0

                _line = df["input"][i].replace('\n', '\\n').replace('"', '\\"')

                f.write(f'{{"input": "{_line}", "p_value": "{p_value}"}}\n')

                if wm_pred:
                    detected += 1
                else:
                    not_detected += 1

    print("\nDetected:", detected)
    print("Not detected:", not_detected)

In [ ]:
calculate_scores('kuditipudi_no_attack.json', "continuation")

 31%|███       | 149/484 [5:36:20<12:37:46, 135.72s/it]

In [ ]:
calculate_scores('kuditipudi_paraphrase.json', "paraphrase")

100%|██████████| 111/111 [3:12:43<00:00, 104.17s/it]


Detected: 77
Not detected: 34


In [ ]:
calculate_scores('kuditipudi_backtranslated.json', "translation")

In [ ]:
calculate_scores('c4_selection.json', "continuation", "kuditipudi_c4_detection.json")